## **Create and store embedding vectors**

In [4]:
ALTER TABLE listings ADD COLUMN listing_vector vector(1536);

UPDATE listings
SET listing_vector = azure_openai.create_embeddings('embedding', description, max_attempts => 5, retry_delay_ms => 500)
WHERE listing_vector IS NULL;

SELECT listing_vector FROM listings LIMIT 1;

: column "listing_vector" of relation "listings" already exists

Total execution time: 00:00:00.099

## **Create the recommendation function**

In [8]:
CREATE OR REPLACE FUNCTION
    recommend_listing(sampleListingId int, numResults int) 
RETURNS TABLE(
    out_listingName text,
    out_listingDescription text,
    out_score real
)
AS $$
DECLARE
    queryEmbedding vector(1536); 
    sampleListingText text; 
BEGIN 
    sampleListingText := (
        SELECT name || ' ' || description
        FROM listings WHERE id = sampleListingId
    ); 

    queryEmbedding := (
        azure_openai.create_embeddings('embedding', sampleListingText, max_attempts => 5, retry_delay_ms => 500)
    );

    RETURN QUERY 
    SELECT
        name::text,
        description,
        (listings.listing_vector <=> queryEmbedding)::real AS score
    FROM
        listings 
    ORDER BY score ASC LIMIT numResults;
END;
$$
LANGUAGE plpgsql;

CREATE FUNCTION

Total execution time: 00:00:00.100

**search for 20 listing recommendations closest to a listing**

In [9]:
select out_listingName, out_score 
   from recommend_listing( 
    (SELECT id from listings limit 1), 
    20); 

(20 row(s) affected)

Total execution time: 00:00:00.341

out_listingname,out_score
Apt on Lake Union,0.0054389243
Private 1BR apartment,0.11501355
Cozy 1BD in Storybook Home,0.11690235
Breeze B&B,0.12103066
Lovely 2 BR Cottage,0.12243493
Lower Queen Anne Studio,0.122817874
Wallingford Craftsman Apt,0.123370305
City Bedroom and Den,0.12375991
Stylish One-Bedroom Apartment,0.12447192
Retro 1BR Greenlake Apt,0.12581536
